In [ ]:
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import geopandas as gpd
import holoviews as hv, geoviews as gv, datashader as ds, panel as pn
import numpy as np 
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
from holoviews.streams import Stream
import pandas as pd
from PIL import Image

hv.extension('bokeh', width=100)
pn.extension()

## reading in geopandas dataframe that is to be edited

In [ ]:
geopandas_filepath = './OG_pictures/kilter_board/new_geopandas_dataframe.shp'
picture_filepath = './OG_pictures/kilter_board.jpeg'

## default geopandas dataframe

In [ ]:
d = {'geometry': ['POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.000, 0.000 0.000, 0.000 0.000))']}
df = pd.DataFrame(d)
gs = gpd.GeoSeries.from_wkt(df['geometry'])
p = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

In [ ]:
polys = hv.Polygons(p).opts(frame_width=800,aspect='equal')

## background image to draw on

In [ ]:
# get image to arrange bounds
img = Image.open(picture_filepath)

# get width and height
width = img.width
height = img.height

# load background with suitable size
pic = hv.RGB.load_image(picture_filepath,bounds=(0,0,width,height))

# produce map from background
#(pic * hv.Bounds((-0.15, -0.4, 0.2, 0))).opts(
#   opts.Bounds(color='orange', line_width=6))

## drawing tool

In [ ]:
poly_draw = streams.PolyDraw(source=polys, drag=True, num_objects=600,
                               show_vertices=True)

# panel to draw on

In [ ]:
# produce map with suitable background and tools
#(pic*polys).opts(
#    opts.Polygons(fill_alpha=0.3, tools=['poly_draw']))#, 'poly_edit', 'tap']))

## button to save the shapes when drawn

In [ ]:
def save_holds(event):
    gdf = gpd.GeoDataFrame(poly_draw.element.data)
    polys=[]
    for i in range(len(gdf)):
        polys.append(Polygon(list(zip(gdf.x[i], gdf.y[i]))))
    gdf = gpd.GeoDataFrame(poly_draw.element.data, geometry = polys)
    gdf.drop(columns=['x','y'], inplace=True)
    #polys = hv.Polygons(gdf)

    mask = gdf.where(gdf.geometry.type == 'GeometryCollection').dropna().index
    gdf_new = gdf.drop(mask)
    gdf_new.to_file(geopandas_filepath)
    confirm_button_range.clicks +=1
    alert.visible = False
    
save_button = pn.widgets.Button(name='Save shapes', width=300, height = 55, align = 'start', button_type = 'success')
save_button.on_click(save_holds)

# Final Plot

In [ ]:
title = title = '<div style="font-size:50px">Drawing Holds</div>'
title = pn.Pane(title, height = 100)

In [ ]:
app = pn.Column(title, save_button," "," ", pn.Row((pic*polys).opts(
    opts.Polygons(fill_alpha=0.3, tools=['poly_draw']))))

In [ ]:
app.servable(title='Drawing Holds')